In [1]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import push_notebook, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, FactorRange, Whisker
from bokeh.models.glyphs import VArea
from bokeh.models.tools import HoverTool
import bokeh.palettes
from bokeh.transform import factor_cmap
import math 

import pandas as pd

df = pd.read_csv('Results/ab10-240sum.csv')
df = df.replace(-1, 0)
print(df)


     ab_duration  runtime image_name ressource_limit  number_of_targets  \
0             10  default   go-httpd         default                  1   
1             10  default   go-httpd         default                  1   
2             10  default   go-httpd         default                  1   
3             10  default   go-httpd         default                  1   
4             10  default   go-httpd         default                  1   
..           ...      ...        ...             ...                ...   
235          240  default   go-httpd         default                  1   
236          240  default   go-httpd         default                  1   
237          240  default   go-httpd         default                  1   
238          240  default   go-httpd         default                  1   
239          240  default   go-httpd         default                  1   

     targetId  repeat  completed_requests  failed_requests  \
0           1       1              20

In [9]:
filterd = df[df['ab_duration'] != 10]

avg = filterd['requests_per_second'].mean()



#print(filterd)
print('Mittelwert über alles: ' + str(avg))

filterd['delta_s'] = filterd['requests_per_second'] - avg
filterd['abw_prozent'] = filterd['delta_s'] * 100 / filterd['requests_per_second']

filterd['mean_over_all']=avg

#print(filterd)

filterd['delta_s_quadrat'] = filterd['delta_s'] * filterd['delta_s']

# std nutzt n-1
grouped_ds_sum = filterd.groupby(['ab_duration'])['requests_per_second'].std().reset_index(name='standartabweichung')
print(grouped_ds_sum)

grouped_req_s_mean = filterd.groupby(['ab_duration'])['requests_per_second'].mean().reset_index(name='mean_requests_per_second')

print(grouped_req_s_mean)

grouped = pd.merge(grouped_req_s_mean, grouped_ds_sum, on='ab_duration')
grouped['konf_intervall_unten'] = grouped['mean_requests_per_second'] - grouped['standartabweichung']  / math.sqrt(40) * 1.6849
grouped['konf_intervall_oben'] = grouped['mean_requests_per_second'] + grouped['standartabweichung']  / math.sqrt(40) * 1.6849
print(grouped)

summe_delta_s_quadrat = filterd['delta_s_quadrat'].sum()
standartabweichung = math.sqrt(summe_delta_s_quadrat/199)
print('standartabweichung: ' + str(standartabweichung))

# Formel t Verteilung: http://www.statistics4u.info/fundstat_germ/cc_confidence_interval.html
# T Verteilungs tabelle: https://www.staff.uni-mainz.de/klenke/vorlesungen/vorl_ws19-2/tabellenwerk.pdf
# Nachschauen:
# n -1 = 39 <- richtig?
# a = 0,005 bei 99,5% Wahrscheinlichkeit
# Werte 38-> 2,712 40-> 2,704 ~ 39 -> 2,7105
# neu: 39 -> 2.7079
# Formel:
# Erwartungswert +- Standartabweichung / sqrt(40) * 2,7105

Mittelwert über alles: 21727.877099999998
   ab_duration  standartabweichung
0           30          239.156890
1           60          156.042139
2           90          130.681180
3          120          132.290688
4          240          124.611921
   ab_duration  mean_requests_per_second
0           30               21679.12150
1           60               21715.47775
2           90               21751.22250
3          120               21720.78325
4          240               21772.78050
   ab_duration  mean_requests_per_second  standartabweichung  \
0           30               21679.12150          239.156890   
1           60               21715.47775          156.042139   
2           90               21751.22250          130.681180   
3          120               21720.78325          132.290688   
4          240               21772.78050          124.611921   

   konf_intervall_unten  konf_intervall_oben  
0          21615.408650         21742.834350  
1          21673.907175

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [5]:

grouped['ab_duration']= grouped['ab_duration'].astype(str)
source = ColumnDataSource(grouped)
duration = source.data['ab_duration'].tolist()
print(duration)

p = figure(
  x_range=duration,
  title = 'apache benchmark duration and bechmark time',
  x_axis_label ='Dauer des ApacheBench Benchmarks',
  y_axis_label ='Requests pro Sekunde',
  plot_width=800
)

color_map = factor_cmap(field_name='ab_duration', palette=bokeh.palettes.d3['Category10'][5], factors=duration)

p.vbar(
    x='ab_duration',
    top='mean_requests_per_second',
    width=0.60,
    source=source,
    color=color_map
)

#p.add_layout(
#    Whisker(source=source, base="ab_duration", upper="konf_intervall_oben", lower="konf_intervall_unten", level="overlay")
#)

output_file("ab_duration.html")
output_notebook()

show(p)

['30', '60', '90', '120', '240']


Loading BokehJS ...

In [11]:
#latex print
grouped.round(4)
printtbl = grouped[['ab_duration', 'mean_requests_per_second', 'konf_intervall_unten', 'konf_intervall_oben', 'standartabweichung']]
print(printtbl.round(2).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &  ab\_duration &  mean\_requests\_per\_second &  konf\_intervall\_unten &  konf\_intervall\_oben &  standartabweichung \\
\midrule
0 &           30 &                  21679.12 &              21615.41 &             21742.83 &              239.16 \\
1 &           60 &                  21715.48 &              21673.91 &             21757.05 &              156.04 \\
2 &           90 &                  21751.22 &              21716.41 &             21786.04 &              130.68 \\
3 &          120 &                  21720.78 &              21685.54 &             21756.03 &              132.29 \\
4 &          240 &                  21772.78 &              21739.58 &             21805.98 &              124.61 \\
\bottomrule
\end{tabular}

